In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import joblib
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report, confusion_matrix

In [18]:
datasets = pd.read_csv('../datasets/train_preprocessed.csv').dropna()
X_train = datasets["content"].astype("string").to_numpy()
y_train = datasets["sentiment"].astype("category").cat.codes.to_numpy()

In [19]:
datasets_t = pd.read_csv('../datasets/test_preprocessed.csv').dropna()
X_test = datasets_t["content"].astype("string").to_numpy()
y_test = datasets_t["sentiment"].astype("category").cat.codes.to_numpy()

In [20]:
vec = CountVectorizer(stop_words='english')
x = vec.fit_transform(X_train).toarray()


In [21]:
x_test = vec.transform(X_test).toarray()

In [22]:
model = MultinomialNB()
model.fit(x, y_train)

MultinomialNB()

In [23]:
model.score(x_test, y_test)

0.315773493244269

In [24]:
pred = model.predict((x_test))

In [25]:
f1 = f1_score(y_test, pred, average='weighted')
f1

0.2696385599978806

In [26]:
report = classification_report(y_test, pred)
print(report)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        36
           1       0.00      0.00      0.00        59
           2       0.00      0.00      0.00       269
           3       0.00      0.00      0.00       250
           4       0.18      0.00      0.01       586
           5       0.33      0.36      0.34      1719
           6       0.44      0.01      0.02       437
           7       0.48      0.33      0.39      1268
           8       0.32      0.39      0.35      2831
           9       0.00      0.00      0.00       504
          10       0.30      0.16      0.21      1703
          11       0.08      0.00      0.00       722
          12       0.29      0.62      0.40      2790

    accuracy                           0.32     13174
   macro avg       0.19      0.14      0.13     13174
weighted avg       0.28      0.32      0.27     13174



/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

In [ ]:
#preprocessed

In [27]:
datasets = pd.read_csv('../datasets/train_preprocessed.csv').dropna()
X_train = datasets["content"].astype("string").to_numpy()
y_train = datasets["sentiment"].astype("category").cat.codes.to_numpy()

In [30]:
datasets_t = pd.read_csv('../datasets/test_preprocessed.csv').dropna()
X_test = datasets_t["content"].astype("string").to_numpy()
y_test = datasets_t["sentiment"].astype("category").cat.codes.to_numpy()

In [28]:
vec = CountVectorizer(stop_words='english')
x = vec.fit_transform(X_train).toarray()

In [31]:
x_test = vec.transform(X_test).toarray()

In [29]:
model = MultinomialNB()
model.fit(x, y_train)

MultinomialNB()

In [32]:
model.score(x_test, y_test)

0.47889782905723394

In [33]:
pred = model.predict((x_test))

In [34]:
f1 = f1_score(y_test, pred, average='weighted')
f1

0.444933095487474

In [35]:
#with augmentation
report = classification_report(y_test, pred)
print(report)

              precision    recall  f1-score   support

           0       1.00      0.03      0.05        36
           1       1.00      0.02      0.03        59
           2       0.95      0.14      0.25       269
           3       0.94      0.13      0.23       250
           4       0.96      0.11      0.20       586
           5       0.51      0.56      0.53      1719
           6       0.91      0.10      0.17       437
           7       0.62      0.48      0.54      1268
           8       0.48      0.58      0.52      2831
           9       0.93      0.06      0.10       504
          10       0.60      0.38      0.46      1703
          11       0.93      0.12      0.22       722
          12       0.39      0.77      0.52      2790

    accuracy                           0.48     13174
   macro avg       0.79      0.27      0.30     13174
weighted avg       0.59      0.48      0.44     13174

